# Custom multi-action agent


In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [22]:
from langchain.agents import AgentExecutor, BaseMultiActionAgent, Tool
from langchain_community.utilities import SerpAPIWrapper

In [3]:
def random_word(query: str) -> str:
    print("\nNow I'm doing this!")
    return "foo"

In [4]:
import os

In [5]:
from google.colab import userdata
SERPAPI_API_KEY=userdata.get('SERPAPI_API_KEY')

In [6]:
os.environ["SERPAPI_API_KEY"] = "da17c0ef2d2f8363db481dace3c36bb05e06a9e4971028ec16bd4c98ede24c0d"

In [16]:
pip install google-search-results

In [8]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="RandomWord",
        func=random_word,
        description="call this to get a random word.",
    ),
]

In [9]:
from typing import Any, List, Tuple, Union

from langchain_core.agents import AgentAction, AgentFinish


class FakeAgent(BaseMultiActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(return_values={"output": "bar"}, log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(return_values={"output": "bar"}, log="")

In [28]:
agent = FakeAgent()

In [11]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [23]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
In 2023, Canada's population jumped by over 1 million people for the first time in the country's history. The population now stands at 39.5 million and is set to pass the 40 million mark later this year.
Now I'm doing this!
foo

> Finished chain.


'bar'

In [13]:
agent_executor.run("How many people live in India as of 2023?")



> Entering new AgentExecutor chain...
1,425,775,850
Now I'm doing this!
foo

> Finished chain.


'bar'

In [24]:
agent_executor.run("what is the capital of india?")



> Entering new AgentExecutor chain...
New Delhi
Now I'm doing this!
foo

> Finished chain.


'bar'

In [46]:
agent_executor.run("what is eiffel tower?")



> Entering new AgentExecutor chain...
['The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower from 1887 to 1889.', 'Eiffel Tower type: Tower in Paris, France.', 'Eiffel Tower entity_type: local_nav.', 'Eiffel Tower kgmid: /m/02j81.', 'Eiffel Tower place_id: ChIJLU7jZClu5kcR4PcOOO6p3I0.', 'Eiffel Tower located_in: Champ de Mars.', 'Eiffel Tower address: Champ de Mars, 5 Av. Anatole France, 75007 Paris, France.', 'Eiffel Tower height: 984′, 1,083′ to tip.', 'Eiffel Tower construction_started: January 26, 1887.', 'Eiffel Tower opened: March 31, 1889.', 'Eiffel Tower architects: Stephen Sauvestre, Émile Nouguier, Maurice Koechlin.', 'Eiffel Tower main_contractor: Compagnie des Etablissements Eiffel.', 'Eiffel Tower floor_count: 4 (3 Accessible + 1 Residual).', 'Eiffel Tower raw_hours: Open ⋅ Closes 10 - 45 PMMonday9 - 30 AM–10 - 45 PMTuesday9 - 30 AM–10 - 45 PMWedne

'bar'

In [26]:
from typing import Any, List, Tuple, Union
from langchain_core.agents import AgentAction, AgentFinish

class CompanyInfoAgent(BaseMultiActionAgent):
    """Custom Agent for Handling Company Information."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decide what action to take.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        input_text = kwargs["input"].lower()  # Convert input text to lowercase for case-insensitive matching
        if "hr portal" in input_text:
            return AgentFinish(return_values={"output": "You can access the HR portal at [HR portal URL]."}, log="")
        elif "employee portal" in input_text:
            return AgentFinish(return_values={"output": "You can access the employee portal at [Employee portal URL]."}, log="")
        elif "company directory" in input_text:
            return AgentFinish(return_values={"output": "You can access the company directory at [Company directory URL]."}, log="")
        # Add more conditions for other company-related queries as needed
        else:
            return AgentFinish(return_values={"output": "I'm sorry, I couldn't understand your request."}, log="")


In [41]:
##Customized agent
from typing import Any, List, Tuple, Union
#from langchain_core.agents import AgentAction, AgentFinish, BaseMultiActionAgent

class CompanyInfoAgent(BaseMultiActionAgent):
    """Custom Agent for Handling Company Information."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decide what action to take.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        input_text = kwargs["input"].lower()  # Convert input text to lowercase for case-insensitive matching
        if "hr portal" in input_text:
            return AgentFinish(return_values={"output": "You can access the portal Kubera for Hr related queries."}, log="")
        elif "employee portal" in input_text:
            return AgentFinish(return_values={"output": "You can access the employee portal Workday for all the queries."}, log="")
        elif "company directory" in input_text:
            return AgentFinish(return_values={"output": "You can access the company directory at [Company directory URL]."}, log="")
        # Add more conditions for other company-related queries as needed
        else:
            return AgentFinish(return_values={"output": "I'm sorry, I couldn't understand your request."}, log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decide what action to take asynchronously.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return self.plan(intermediate_steps=intermediate_steps, **kwargs)


In [42]:
# Instantiate the custom agent
agent = CompanyInfoAgent()



In [43]:
input_text = "How do I access the HR portal?"

result = agent.plan(intermediate_steps=[], input=input_text)

if isinstance(result, AgentFinish):
    print(result.return_values["output"])
else:
    # Handle unexpected result
    print("Unexpected result from agent.")

You can access the portal Kubera for Hr related queries.


In [44]:
input_text = "How do I access the employee portal?"

result = agent.plan(intermediate_steps=[], input=input_text)

if isinstance(result, AgentFinish):
    print(result.return_values["output"])
else:
    # Handle unexpected result
    print("Unexpected result from agent.")

You can access the employee portal Workday for all the queries.


In [ ]:
###text summarization agent

In [45]:
class TextSummarizationAgent:
    """Agent for Text Summarization."""

    def summarize_text(self, input_text, ratio=0.2):
        """Summarize input text.

        Args:
            input_text (str): The input text to be summarized.
            ratio (float): The ratio of sentences to be included in the summary (default is 0.2).

        Returns:
            summary (str): The summarized text.
        """
        try:
            summary = summarize(input_text, ratio=ratio)
            return summary
        except Exception as e:
            print("Error occurred during text summarization:", e)
            return None

    def summarize_eiffel_tower(self):
        """Summarize information about the Eiffel Tower."""
        eiffel_tower_info = """
            The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
            It is named after the engineer Gustave Eiffel, whose company designed and built the tower.
            Completed in 1889, it has become a global cultural icon of France and one of the most recognizable structures in the world.
            The tower is 324 meters tall, about the same height as an 81-story building.
            """

        return eiffel_tower_info

# Instantiate the text summarization agent
summarization_agent = TextSummarizationAgent()

eiffel_tower_summary = summarization_agent.summarize_eiffel_tower()

print("Summary about the Eiffel Tower:")
print(eiffel_tower_summary)


Summary about the Eiffel Tower:

            The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. 
            It is named after the engineer Gustave Eiffel, whose company designed and built the tower.
            Completed in 1889, it has become a global cultural icon of France and one of the most recognizable structures in the world.
            The tower is 324 meters tall, about the same height as an 81-story building.
            


In [ ]:
import sqlite3

class DatabaseAgent:
    """Agent for interacting with a SQLite database."""

    def __init__(self, db_file):
        """Initialize the database connection."""
        self.db_file = db_file
        self.connection = None
        self.cursor = None

    def connect(self):
        """Establish a connection to the SQLite database."""
        try:
            self.connection = sqlite3.connect(self.db_file)
            self.cursor = self.connection.cursor()
            print("Database connection established.")
        except sqlite3.Error as e:
            print("Error connecting to the database:", e)

    def close(self):
        """Close the database connection."""
        if self.connection:
            self.connection.close()
            print("Database connection closed.")

    def fetch_data(self, query):
        """Fetch data from the database based on the provided query."""
        try:
            self.cursor.execute(query)
            data = self.cursor.fetchall()
            return data
        except sqlite3.Error as e:
            print("Error fetching data from the database:", e)
            return None

if __name__ == "__main__":
    # Initialize the database agent
    db_agent = DatabaseAgent("test.db")

    db_agent.connect()

    user_input = "Eiffel Tower"

    # Fetch data related to user input
    query = f"SELECT * FROM landmarks WHERE name = '{user_input}'"
    result = db_agent.fetch_data(query)


    if result:
        print("Data related to user input:", result)
    else:
        print("No data found.")

    db_agent.close()


In [48]:
#"CompanyInfoAgent" to use information collected by another agent and answer a query,
#we can design it to interact with the database or any other data source where relevant information is stored

In [ ]:
class CompanyInfoAgent(BaseMultiActionAgent):
    """Custom Agent for Handling Company Information."""

    def __init__(self, db_agent):
        """Initialize the CompanyInfoAgent with a database agent."""
        super().__init__()
        self.db_agent = db_agent

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decide what action to take.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        input_text = kwargs["input"].lower()
        if "hr portal" in input_text:
            # Use the database agent to fetch information about the HR portal
            hr_portal_info = self.db_agent.fetch_hr_portal_info()
            return AgentFinish(return_values={"output": hr_portal_info}, log="")
        elif "employee portal" in input_text:
            # Use the database agent to fetch information about the employee portal
            employee_portal_info = self.db_agent.fetch_employee_portal_info()
            return AgentFinish(return_values={"output": employee_portal_info}, log="")
        elif "company directory" in input_text:
            # Use the database agent to fetch information about the company directory
            company_directory_info = self.db_agent.fetch_company_directory_info()
            return AgentFinish(return_values={"output": company_directory_info}, log="")
        # we can add the more conditions if required here
        else:
            return AgentFinish(return_values={"output": "I'm sorry, I couldn't understand your request."}, log="")
